# IMPORTING NECESSARY LIBRARIES

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

from sklearn.metrics import r2_score

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv("day.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'day.csv'

In [ ]:
df.head()

In [ ]:
df.info()

Our dataset does not contain any null values as described by the above prompt




day.csv have the following fields:

	- instant: record index
	- dteday : date
	- season : season (1:spring, 2:summer, 3:fall, 4:winter)
	- yr : year (0: 2018, 1:2019)
	- mnth : month ( 1 to 12)
	- holiday : weather day is a holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)
	- weekday : day of the week
	- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
	+ weathersit :
		- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
		- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
		- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
		- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
	- temp : temperature in Celsius
	- atemp: feeling temperature in Celsius
	- hum: humidity
	- windspeed: wind speed
	- casual: count of casual users
	- registered: count of registered users
	- cnt: count of total rental bikes including both casual and registered

In [ ]:
df.describe()

# DATA PREPROCESSING

And its already mentioned that "casual" and "registered" users are not the target variables so I'm going to drop those columns as well

In [ ]:
df.drop(["casual","registered"], axis = 1, inplace = True)

In [ ]:
df.head()

Since we are dealing with data if only year 2018, 2019 I want to check for outliers in the dteday column

In [ ]:
# a summary on years

""".split("-") will split the date into 3 different parts x.split[0]: date,
x.split[1]: month, x.split[2]: year"""
df["dteday"].apply(lambda x: int(x.split("-")[2])).describe()


In [ ]:
# a summary on years
df["dteday"].apply(lambda x: int(x.split("-")[0])).describe()

#since we already have a month column there is o actual need to verify that

In [ ]:
# I want to rename some columns for beter uncerstanding
df.rename(columns = {"yr":"year", "mnth":"month", "hum":"humidity", "cnt":"total_count"}, inplace=True)

In [ ]:
# making a local copy of our dataframe
dataset = df.copy()

In [ ]:
# And I want to check for some duplicate rows in our data
dataset.shape[0]
print("Before Dropping Duplicates \n")
print(f"There are '{dataset.shape[0]}' rows and '{dataset.shape[1]}' columns in our dataset \n")
#dropping duplicates
dataset.drop_duplicates(inplace=True)
print("\nAfter Dropping Duplicates")
print(f"There are '{dataset.shape[0]}' rows and '{dataset.shape[1]}' columns in our dataset \n")

#DATA VISUALIZATION

##Pairplot to see relation among variables

In [ ]:
#scatter plot shows us the relation between two variables, so I'm using pairplot to intricacies between variables
sns.pairplot(dataset[["temp","atemp","humidity","windspeed","weathersit","total_count"]])
plt.show()

### OBSERVATION :

1. The relation between temp and atemp is highly correlated and this shows that there is a linear relationship between them.
2. It can be said that when the humidity is above 40, more people prefer to take rented bikes.
3. People are less likely to rent bikes when the wind speed is above 25.

##Season wise Distribution of total Rentals

In [ ]:
sns.barplot(x="season", y="total_count", data=dataset, hue="season")
plt.xlabel('Season')
plt.ylabel('Total Rentals')
plt.title('Total rentals by season')

### NOTE:

But before we move further with the visualization I want to map values for some categorical features

Categorical Features : "Season, Year, Month, Weekday, Weathersit"

Since we are mapping string values to the categorical features we need to change their dtype

In [ ]:
#mapping correct values for the column "Season"
season_list = {1:"spring",2:"summer",3:"fall", 4:"winter"}
dataset.season = dataset.season.map(season_list)

In [ ]:
#mapping correct values for the column "Year"
dataset["year"] = dataset["year"].map({0:2018, 1:2019})

In [ ]:
#mapping correct values for the column "Month"
dataset["month"] = dataset["month"].map({1:"Jan",2:"Feb",3:"Mar",4:"Apr",\
                                         5:"May", 6:"jun",7:"Jul",8:"Aug",\
                                         9:"Sep", 10:"Oct",11:"Nov",12:"Dec"})

In [ ]:
#mapping correct values for the column "Weekday"
dataset["weekday"] = dataset["weekday"].map({0:"sun",1:"mon",2:"tue", 3:"wed", 4:"thr",\
                                             5:"fri",6:"sat"})

In [ ]:
#mapping correct values for the column "Weathersit"
dataset["weathersit"] = dataset["weathersit"].map({1:"clear",2:"misty",3:"light snow & rain",\
                                           4:"heavy snow & rain"})

In [ ]:
dataset.head()

##3. Total Count by Month

In [ ]:
mean_counts = df.groupby("month")["total_count"].mean()

# Create a bar plot using Seaborn
plt.figure(figsize=(10, 6))
ax = sns.barplot(x=mean_counts.index, y=mean_counts.values)

x_label = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
plt.xticks(range(len(x_label)), x_label, rotation=0)

plt.xlabel("Month")
plt.ylabel("Mean Total Count")
plt.title("Mean Total Count by Month")

# Add values on top of bars
for p in ax.patches:
    ax.annotate(f"{p.get_height():.0f}", (p.get_x() + p.get_width() / 2., p.get_height()),
                ha="center", va="bottom", fontsize=10)

plt.show()

In [ ]:
sns.boxplot(data=dataset, x= "month", y="total_count", hue = "month")
plt.show()

In [ ]:
sns.boxplot(data=dataset, x="weekday", y="total_count", hue = "weekday")
plt.show()

##4. Season vs Weekday

In [ ]:
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['weekday'].value_counts()
    for x_label, grp in dataset.groupby('season')
})
sns.heatmap(df_2dhist, cmap='viridis', annot =True)
plt.xlabel('season')
_ = plt.ylabel('weekday')

##5. Average Temperature vs Total Rentals

In [ ]:
plt.scatter(df['temp'], df['total_count'], c=df['temp'], cmap='viridis')
plt.xlabel('Temperature')
plt.ylabel('Total Rentals')
plt.title('Average Temperature vs Total Rentals')

In [ ]:
sns.barplot(data=dataset.groupby("year")["total_count"].sum())
plt.show()

In [ ]:
sns.barplot(data=dataset.groupby("holiday")["total_count"].sum())
plt.show()

In [ ]:
sns.barplot(data=dataset.groupby("weekday")["total_count"].sum())
plt.show()

## Correlation Heatmap

In [ ]:
dataset.info()

In [ ]:
sns.heatmap(dataset[["temp","atemp","humidity","workingday","windspeed","total_count"]].corr(), annot=True, fmt='.2f')
plt.title("Correlation Between Numerical Variables")
plt.show()

## Observation :

There is a very high correlation betwwen temp and atemp, using both of these features in our linear regression model will introduce the conept of multicollinearity.

In [ ]:
dataset.drop("temp",axis =1 , inplace = True)

In [ ]:
dataset["total_count"].plot.density()

#DATA PREPERATION

I'm going to create dummy variablkes for all the categorical features

In [ ]:
# creating dummy variavles for month, weekdays, weathersi and season columns
month_dummy = pd.get_dummies(dataset.month, drop_first = True)
weekday_dummy = pd.get_dummies(dataset.weekday, drop_first=True)
weathersit_dummy= pd.get_dummies(dataset.weathersit, drop_first=True)
season_dummy = pd.get_dummies(dataset.season, drop_first=True)

In [ ]:
#converting bool to int
month_dummy = month_dummy.astype(int)
weekday_dummy = weekday_dummy.astype(int)
weathersit_dummy= weathersit_dummy.astype(int)
season_dummy = season_dummy.astype(int)

In [ ]:
#merging these column sback into datset(local copy df)
#new version of dataset column, dataset_new
dataset_new = pd.concat([dataset, month_dummy, weekday_dummy, weathersit_dummy, season_dummy], axis= 1)

In [ ]:
dataset_new.info()

In [ ]:
#dteday and instant does not prive any value so I'm going to drop those features.
# And since we have created dummy variables we dont need "Season","Weekday","month", weathersit" columns
dataset_new.drop(["instant","dteday","season","weekday","month","weathersit"], axis = 1, inplace =True)

# SPLITTING THE DATA

In [ ]:
#specify the random seed
np.random.seed(100)

#usually we form 4 series variable x_train, y_trainm,m x_test and y_test  here I'll use two dataframes for simplicity
dataset_train, dataset_test = train_test_split(dataset_new, train_size=0.7, random_state = 100)

print(f" The dataset_train has {dataset_train.shape[0]} rows and {dataset_train.shape[1]} columns.")
print(f" The dataset_test has {dataset_test.shape[0]} rows and {dataset_test.shape[1]} columns.")

In [ ]:
dataset_train.head()

In [ ]:
#rescaling features
scaler = StandardScaler()
num_features = ["atemp","humidity","windspeed"]

#fit_transform
dataset_train[num_features] = scaler.fit_transform(dataset_train[num_features])
dataset_train.head()

In [ ]:
#Heatmap to see which feature is highly correlated
plt.figure(figsize=(18, 14))
sns.heatmap(data=dataset_new.corr(), annot=True, fmt=".2f")
plt.show()

#TRAINING THE MODEL

In [ ]:
y_train = dataset_train.pop('total_count')
x_train = dataset_train

## Building 1st model with all variables

### Test on Training Set

In [ ]:
x_train_sm = sm.add_constant(x_train)
lr = sm.OLS(y_train, x_train_sm)
lr_model = lr.fit()
lr_model.summary()

### Checking its VIF

In [ ]:
vif = pd.DataFrame()
vif["Features"] = x_train.columns
vif["VIF"] = [variance_inflation_factor(x_train.values, i) for i in range(x_train.shape[1])]
vif["VIF"] = round(vif["VIF"], 2)
vif = vif.sort_values(by="VIF", ascending = False)
vif

## 2ND MODEL

In [ ]:
#dropping the"workingday" column although it's P>|t| is 0.00 but it is shure introducing a lot of multicolinearity
x = x_train.drop("workingday", axis = 1)

#creating new model on "x"
x_train_sm = sm.add_constant(x)
lr = sm.OLS(y_train, x_train_sm)
lr_model = lr.fit()
lr_model.summary()

In [ ]:
vif = pd.DataFrame()
vif["Features"] = x.columns
vif["VIF"] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]
vif["VIF"] = round(vif["VIF"], 2)
vif = vif.sort_values(by="VIF", ascending = False)
vif

## 3RD MODEL

In [ ]:
#dropping the"Oct" column due to high P>|t| and VIF values

x = x.drop("Oct", axis = 1)

#creating new model on "x"
x_train_sm = sm.add_constant(x)
lr = sm.OLS(y_train, x_train_sm)
lr_model = lr.fit()
lr_model.summary()

In [ ]:
vif = pd.DataFrame()
vif["Features"] = x.columns
vif["VIF"] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]
vif["VIF"] = round(vif["VIF"], 2)
vif = vif.sort_values(by="VIF", ascending = False)
vif

## 4TH MODEL

In [ ]:
#dropping the "Mar" column due to high P>|t| and VIF values

x = x.drop("Mar", axis = 1)

#creating new model on "x"
x_train_sm = sm.add_constant(x)
lr = sm.OLS(y_train, x_train_sm)
lr_model = lr.fit()
lr_model.summary()

In [ ]:
vif = pd.DataFrame()
vif["Features"] = x.columns
vif["VIF"] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]
vif["VIF"] = round(vif["VIF"], 2)
vif = vif.sort_values(by="VIF", ascending = False)
vif

## 5TH MODEL

In [ ]:
#dropping the "sat" column due to high P>|t| and VIF values

x = x.drop("sat", axis = 1)

#creating new model on "x"
x_train_sm = sm.add_constant(x)
lr = sm.OLS(y_train, x_train_sm)
lr_model = lr.fit()
lr_model.summary()

In [ ]:
vif = pd.DataFrame()
vif["Features"] = x.columns
vif["VIF"] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]
vif["VIF"] = round(vif["VIF"], 2)
vif = vif.sort_values(by="VIF", ascending = False)
vif

## 6TH MODEL

In [ ]:
#dropping the "wed" column due to high P>|t| and VIF values

x = x.drop("wed", axis = 1)

#creating new model on "x"
x_train_sm = sm.add_constant(x)
lr = sm.OLS(y_train, x_train_sm)
lr_model = lr.fit()
lr_model.summary()

In [ ]:
vif = pd.DataFrame()
vif["Features"] = x.columns
vif["VIF"] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]
vif["VIF"] = round(vif["VIF"], 2)
vif = vif.sort_values(by="VIF", ascending = False)
vif

## 7TH MODEL

In [ ]:
#dropping the "summer" column due to high P>|t| and VIF values

x = x.drop("summer", axis = 1)

#creating new model on "x"
x_train_sm = sm.add_constant(x)
lr = sm.OLS(y_train, x_train_sm)
lr_model = lr.fit()
lr_model.summary()

In [ ]:
vif = pd.DataFrame()
vif["Features"] = x.columns
vif["VIF"] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]
vif["VIF"] = round(vif["VIF"], 2)
vif = vif.sort_values(by="VIF", ascending = False)
vif

# TESTING THE OUTPUT - RESIDUAL ANALYSIS

In [ ]:
# Residual analysis
y_train_pred = lr_model.predict(x_train_sm)
res = y_train - y_train_pred

# plot the residual
fig = plt.figure()
sns.distplot(res)
fig.suptitle('Error Terms', fontsize = 20)
plt.xlabel('Errors', fontsize = 18)
plt.show()

## OBSERVATION - ERROR TERMS
Errors terms are normally distributed which indicate that the model's assumptions are met

In [ ]:
# Homoscedasticity
plt.scatter(res, y_train_pred)
plt.show()

## OBSERVATION - ERROR TERMS
There is no pattern observed in the above graph, hence Homoscedasticity assumption is satisfied

#PREDICTIONG AND EVALUATION

In [ ]:
# Rescaling on numeric variables for Test set
num_vars = ['atemp','humidity','windspeed']

# Fit on data
dataset_test[num_vars] = scaler.transform(dataset_test[num_vars])
dataset_test.head()

In [ ]:
y_test = dataset_test.pop("total_count")
x_test = dataset_test

In [ ]:
dataset_test.head()

In [ ]:
#adding a constant
x_test_sm = sm.add_constant(x_test)
x_test_sm.head()

In [ ]:
# dropping the features which were not in the final model
x_test_sm.drop(["workingday","Oct","Mar","sat","wed","summer"], axis=1, inplace=True)
x_test_sm.head()

In [ ]:
#prediciting the model
y_test_pred = lr_model.predict(x_test_sm)

In [ ]:
#evaluating
r2_score(y_true= y_test, y_pred=y_test_pred)